In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [64]:
# Make sure to have 'books.data' in your directory! It will be imported as 'books_df'
import pickle
with open('books.data', 'rb') as filehandle:  
    books_df = pickle.load(filehandle)



import pickle
with open('word_len.data', 'rb') as filehandle:  
    word_mean = pickle.load(filehandle)



import csv
ratings_csv = csv.reader(open('ratings.txt'))
ratings = []
for row in ratings_csv:
    ratings.append(row[0])
ratings = list(map(lambda x: float(x), ratings))
ratings


books_stats = {'tokens':books_df,'ratings':ratings,'pos_perc':pos_perc,'neg_perc':neg_perc,
              'word_count':word_count,'ave_len':ave_len}


books_stats['means'][:3]

import numpy as np
import pandas as pd
books_stats['ratings'] = pd.Series(books_stats['ratings'])


from nltk.stem import PorterStemmer
ps = PorterStemmer()

pos_csv = csv.reader(open('positive-words.txt'))
neg_csv = csv.reader(open('negative-words.txt'))
pos = []
neg = []
for row in pos_csv:
    pos.append(row[0])
for row in neg_csv:
    neg.append(row[0])



pos[:5]
neg[:5]
for s in pos:
    if s in neg:
        print(s)


for i in range(len(books_stats['tokens'])):
    if i % 10 == 0:
        print(i)
    books_stats['tokens'][i]['positive'] = pd.Series(map(lambda x: 1 if x in pos else 0, 
                        list(books_stats['tokens'][i]['full'])), index = books_stats['tokens'][i].index)
    books_stats['tokens'][i]['negative'] = pd.Series(map(lambda x: 1 if x in neg else 0, 
                        list(books_stats['tokens'][i]['full'])), index = books_stats['tokens'][i].index)



list(books_stats['tokens'][0].index)

pd.Series(map(lambda x: 1 if (x in pos) else 0, list(books_stats['tokens'][0]['word'])),
          index = books_stats['tokens'][0].index)


books_stats['tokens'][0]

for i in range(len(books_stats['tokens'])):
    books_stats['tokens'][i]['full'] = pd.Series(map(lambda x: str(x).lower(), list(books_stats['tokens'][i].index)),
                                                 index = books_stats['tokens'][i].index)



word_count = pd.Series(map(lambda x: len(x), books_stats['tokens']))
pos_count = pd.Series(map(lambda i: sum(books_stats['tokens'][i]['positive']), range(len(books_stats['tokens']))))
neg_count = pd.Series(map(lambda i: sum(books_stats['tokens'][i]['negative']), range(len(books_stats['tokens']))))
pos_perc = pd.Series(map(lambda i: pos_count[i]/word_count[i], range(len(books_stats['tokens']))))
neg_perc = pd.Series(map(lambda i: neg_count[i]/word_count[i], range(len(books_stats['tokens']))))



sum(pos_perc < neg_perc)

temp_arr = np.array([books_stats['ratings'],pos_perc,neg_perc])

temp_arr = temp_arr[:,temp_arr[0,:] != 0]

np.mean(temp_arr[0,temp_arr[1,:] < temp_arr[2,:]])

np.mean(temp_arr[0,temp_arr[1,:] > temp_arr[2,:]])

import pickle
with open('books_stats.data', 'wb') as filehandle:  
    # store the data as binary data stream
    pickle.dump(books_stats, filehandle)



books_stats.keys()

for i in range(len(books_stats['tokens'])):
    books_stats['tokens'][i]['length'] = pd.Series(map(lambda x: len(x), books_stats['tokens'][i]['full'])
                                                  , index = books_stats['tokens'][i].index)



ave_len = pd.Series(map(lambda x: books_stats['tokens'][x]['length'].mean(), range(len(books_stats['tokens']))))


books_stats['ave_len']

pos_perc

########## Basic model ############
X = np.array(pd.DataFrame([word_count,ave_len,pos_perc,neg_perc])).transpose()
Y = np.array(books_stats['ratings'])


X = X[Y != 0,:]
Y = Y[Y != 0]
Y = np.array( list(map(lambda y: 1 if y > 4 else 0, Y)) )


idx = np.random.permutation(len(Y))
t = int(.8*len(Y))
X_train = X[idx[:t],:]
X_test = X[idx[t:],:]
Y_train = Y[idx[:t]]
Y_test = Y[idx[t:]]


X_train.shape

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 6174)
# Train the model on training data
rf.fit(X_train, Y_train);



Y_train_pred = rf.predict(X_train)
Y_test_pred = rf.predict(X_test)



from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_test_pred, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.figure(figsize = (15,12))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

sum(Y_test)


sum((Y_train_pred - Y_train)**2)/len(Y_train)
sum((np.mean(Y_train) - Y_train)**2)/len(Y_train)
sum((Y_test_pred - Y_test)**2)/len(Y_test)
sum((np.mean(Y_test) - Y_test)**2)/len(Y_test)



Y_train_pred

from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

w2v_model = gensim.models.Word2Vec(thing, min_count = 1, size = 100, window = 10)

import pickle
with open('w2v_model.data', 'wb') as filehandle:  
    # store the data as binary data stream
    pickle.dump(w2v_model, filehandle)




thing[1]

w2v_model.wv.vocab

w2v_model.similarity('alic','wonderland')



len(books_stats['tokens'][0])

((1136537/2571.0)*3.41)/60

import nltk
nltk.download('gutenberg')



gutenberg.words(gutenberg.fileids()[0])[-1]

